In [1]:
import pandas as pd
import re

In [2]:
call_data = pd.read_csv("data/call_data_from_CAD.csv")

In [3]:
call_data.head()

,Unnamed: 0,IncidentNumber,Call_Created_Time,Call_First_Dispatched_Time,Call_First_On_Scene,Call_Cleared,Call_Zipcode,Beat,Call_Source,Call_Priority,InitialIncidentTypeDescription,IsPrimary,PrimaryUnitCallSign,RespondingUnitCallSign,Unit_Dispatched_Time,Unit_OnScene_Time,Unit_Cleared_Time,Disposition
0,1,OR-2016-01-01-16000001,1/1/16 0:00,1/1/16 0:04,1/1/16 0:09,1/1/16 0:54,97402.0,EP05,E911,3,ASSAULT,1,5E57,5E57,1/1/16 0:04,1/1/16 0:09,1/1/16 0:54,ADVISED
1,2,OR-2016-01-01-16000001,1/1/16 0:00,1/1/16 0:04,1/1/16 0:09,1/1/16 0:54,97402.0,EP05,E911,3,ASSAULT,0,5E57,4X40,1/1/16 0:09,1/1/16 0:09,1/1/16 0:46,ADVISED
2,3,OR-2016-01-01-16000001,1/1/16 0:00,1/1/16 0:04,1/1/16 0:09,1/1/16 0:54,97402.0,EP05,E911,3,ASSAULT,0,5E57,4E53,1/1/16 0:04,1/1/16 0:12,1/1/16 0:51,ADVISED
3,4,OR-2016-01-01-16000003,1/1/16 0:00,1/1/16 0:00,1/1/16 0:00,1/1/16 0:02,97401.0,EP02,SELF,6,TRAFFIC STOP,1,5T81,5T81,1/1/16 0:00,1/1/16 0:00,1/1/16 0:02,ADVISED
4,5,OR-2016-01-01-16000004,1/1/16 0:02,1/1/16 0:04,1/1/16 0:04,1/1/16 0:18,97401.0,EP02,E911,3,CHECK WELFARE,0,3X90,3F61,1/1/16 0:04,NaN,1/1/16 0:08,ASSISTED


In [4]:
pattern = r"\b(1J77|3J79|3J78|3J77|4J79|3J81|3J76|2J28|2J29|CAHOOT|CAHOT|CAHO)\b"

def unit_call_sign(call_sign):
    if pd.isna(call_sign):
        return "unknown"
    elif re.fullmatch(pattern, call_sign):
        return "cahoots"
    else:
        return "police"

call_data["call_unit"] = call_data["PrimaryUnitCallSign"].apply(unit_call_sign)
call_data["responding_unit"] = call_data["PrimaryUnitCallSign"].fillna("").apply(
    lambda x: "cahoots" if re.fullmatch(pattern, x) else "police"
)

In [5]:
call_data = call_data.drop(["Call_Created_Time", "Call_First_Dispatched_Time", "Call_First_On_Scene", "Call_Cleared", "Beat", 
                "Call_Source", "IsPrimary", "PrimaryUnitCallSign", "RespondingUnitCallSign", "Unit_Dispatched_Time",
                "Unit_OnScene_Time", "Unit_Cleared_Time"], axis=1)

In [6]:
call_data = call_data[call_data["responding_unit"] == "cahoots"]

In [7]:
#call_data["Call_Zipcode"].isna().sum()
#call_data.isna().sum()
call_data = call_data.dropna()

In [8]:
#call_data["Call_Zipcode"].unique()

In [9]:
call_data["Call_Zipcode"] = call_data["Call_Zipcode"].astype(int)
call_data.head()

,Unnamed: 0,IncidentNumber,Call_Zipcode,Call_Priority,InitialIncidentTypeDescription,Disposition,call_unit,responding_unit
18,19,OR-2016-01-01-16000016,97402,7,TRANSPORT,DISREGARD,cahoots,cahoots
27,28,OR-2016-01-01-16000033,97402,7,TRANSPORT,DISREGARDED BY DISPATCH,cahoots,cahoots
43,44,OR-2016-01-01-16000044,97402,7,INTOXICATED SUBJECT,UNABLE TO LOCATE,cahoots,cahoots
63,64,OR-2016-01-01-16000069,97401,7,ASSIST PUBLIC- POLICE,DISREGARDED BY DISPATCH,cahoots,cahoots
151,152,OR-2016-01-01-16000148,97402,7,ASSIST PUBLIC- POLICE,GONE ON ARRIVAL,cahoots,cahoots


In [10]:
call_data["Call_Zipcode"].unique()

array([97402, 97401, 97403, 97404, 97405, 97408, 97477, 97424, 97448,
       97437, 97419])

In [11]:
call_data_clean = call_data

In [12]:
race_ethnicity_data = pd.read_csv("data/race_and_ethnicity_data.csv", header = [1, 2])
race_ethnicity_data.columns = [f"{a}_{b}" if "Unnamed" not in str(a) else b.lower() for a, b in race_ethnicity_data.columns]
race_ethnicity_data.head()

,unnamed: 0_level_1,United States_Value,error,Oregon_Value,error,"Lane County, OR_Value",error,"Eugene-Springfield, OR Metro Area_Value",error,97324_Value,...,97489_Value,error,97490_Value,error,97492_Value,error,97493_Value,error,97498_Value,error
0,Total:,332387550,NaN,4238714,NaN,382628,NaN,382628,NaN,1129,...,896,359,476,288,1058,505,466,128,2141,324
1,Not Hispanic or Latino:,269255940,1579.0,3633247,NaN,343584,NaN,343584,NaN,1129,...,700,224,467,286,1036,503,466,128,2000,316
2,White alone,193338270,30069.0,3065611,3460.0,301961,795.0,301961,795.0,999,...,658,220,389,244,1007,497,430,114,1797,266
3,Black or African American alone,39981608,43649.0,77800,1828.0,4060,373.0,4060,373.0,0,...,0,13,0,13,5,9,0,13,5,7
4,American Indian and Alaska Native alone,1751338,8345.0,28235,1397.0,2532,408.0,2532,408.0,4,...,0,13,0,13,22,36,0,13,0,13


In [13]:
race_ethnicity_data_clean = race_ethnicity_data[["unnamed: 0_level_1", "97402_Value", "97401_Value", "97403_Value", "97404_Value", "97405_Value", "97408_Value", 
"97477_Value", "97424_Value", "97448_Value", "97437_Value", "97419_Value"]]

race_ethnicity_data_clean = race_ethnicity_data_clean.rename(columns={"unnamed: 0_level_1": "race_and_ethnicity"})
race_ethnicity_data_clean.head()

,race_and_ethnicity,97402_Value,97401_Value,97403_Value,97404_Value,97405_Value,97408_Value,97477_Value,97424_Value,97448_Value,97437_Value,97419_Value
0,Total:,54199,48186,14508,33356,46870,15297,37837,18881,14629,2462,916
1,Not Hispanic or Latino:,45186,43438,13247,30106,43370,13971,32752,17491,13407,2324,825
2,White alone,38923,36145,10761,26731,38237,11819,29186,15397,12339,2243,702
3,Black or African American alone,985,654,326,428,311,424,321,21,78,32,0
4,American Indian and Alaska Native alone,397,172,54,183,278,100,168,132,69,0,0


In [14]:
sex_age_data = pd.read_csv("data/sex_and_age_data.csv", header = [1, 2])
sex_age_data.columns = [f"{a}_{b}" if "Unnamed" not in str(a) else b.lower() for a, b in sex_age_data.columns]
sex_age_data.head()

,unnamed: 0_level_1,United States_Value,error,Oregon_Value,error,"Lane County, OR_Value",error,"Eugene-Springfield, OR Metro Area_Value",error,97324_Value,...,97489_Value,error,97490_Value,error,97492_Value,error,97493_Value,error,97498_Value,error
0,Total:,332387550,NaN,4238714,NaN,382628,NaN,382628,NaN,1129,...,896,359,476,288,1058,505,466,128,2141,324
1,Male:,164545090,6966.0,2113849,981.0,189067,65.0,189067,65.0,584,...,507,205,266,177,570,273,230,66,969,178
2,Under 5 years,9688436,4185.0,109030,503.0,8618,25.0,8618,25.0,10,...,0,13,14,21,45,76,0,13,3,11
3,5 to 9 years,10296243,22450.0,121246,1925.0,9771,644.0,9771,644.0,14,...,0,13,6,9,4,7,0,13,79,62
4,10 to 14 years,11032019,22077.0,131064,1956.0,10360,643.0,10360,643.0,19,...,79,74,9,17,38,44,20,31,65,49


In [15]:
sex_age_data_clean = sex_age_data[["unnamed: 0_level_1", "97402_Value", "97401_Value", "97403_Value", "97404_Value", "97405_Value", "97408_Value", 
"97477_Value", "97424_Value", "97448_Value", "97437_Value", "97419_Value"]]

sex_age_data_clean = sex_age_data_clean.rename(columns={"unnamed: 0_level_1": "sex_and_age"})
sex_age_data_clean.head()

,sex_and_age,97402_Value,97401_Value,97403_Value,97404_Value,97405_Value,97408_Value,97477_Value,97424_Value,97448_Value,97437_Value,97419_Value
0,Total:,54199,48186,14508,33356,46870,15297,37837,18881,14629,2462,916
1,Male:,27643,23185,6230,16328,23263,7381,18721,9626,7223,1127,486
2,Under 5 years,1497,832,224,925,916,253,1115,412,412,44,26
3,5 to 9 years,1678,612,150,901,1042,461,1030,642,524,58,4
4,10 to 14 years,1807,694,106,787,1175,534,1204,589,314,116,23


In [16]:
education_data = pd.read_csv("data/education_by_sex_data.csv", header = [1, 2])
education_data.columns = [f"{a}_{b}" if "Unnamed" not in str(a) else b.lower() for a, b in education_data.columns]
education_data.head()

,unnamed: 0_level_1,United States_Value,error,Oregon_Value,error,"Lane County, OR_Value",error,"Eugene-Springfield, OR Metro Area_Value",error,97324_Value,...,97489_Value,error,97490_Value,error,97492_Value,error,97493_Value,error,97498_Value,error
0,Total:,228434660,11047,3021010,716,266887,50,266887,50,938,...,656,220,343,214,835,452,362,110,1687,238
1,Male:,111381464,7804,1489634,774,130479,43,130479,43,503,...,343,130,189,114,433,245,204,70,734,119
2,No schooling completed,2122372,17274,22196,1308,1304,290,1304,290,0,...,0,13,0,13,1,3,0,13,8,13
3,Nursery to 4th grade,627694,10889,6751,845,333,156,333,156,0,...,0,13,0,13,4,8,0,13,0,13
4,5th and 6th grade,1295383,19276,13425,1216,771,297,771,297,0,...,33,54,0,13,0,13,0,13,0,13


In [17]:
education_data_clean = education_data[["unnamed: 0_level_1", "97402_Value", "97401_Value", "97403_Value", "97404_Value", "97405_Value", "97408_Value", 
"97477_Value", "97424_Value", "97448_Value", "97437_Value", "97419_Value"]]

education_data_clean = education_data_clean.rename(columns={"unnamed: 0_level_1": "education"})
education_data_clean.head()

,education,97402_Value,97401_Value,97403_Value,97404_Value,97405_Value,97408_Value,97477_Value,97424_Value,97448_Value,97437_Value,97419_Value
0,Total:,37942,29347,5996,24533,34393,10899,26799,13250,10638,1680,724
1,Male:,18876,14455,2739,12116,17372,5035,12593,6303,5270,778,345
2,No schooling completed,213,90,14,243,102,29,224,61,14,2,0
3,Nursery to 4th grade,49,12,15,15,0,0,50,0,174,0,0
4,5th and 6th grade,155,31,0,173,13,0,253,18,23,0,0


In [18]:
income_data = pd.read_csv("data/income_data.csv", header = [1, 2])
income_data.columns = [f"{a}_{b}" if "Unnamed" not in str(a) else b.lower() for a, b in income_data.columns]
income_data.head()

,unnamed: 0_level_1,United States_Value,error,Oregon_Value,error,"Lane County, OR_Value",error,"Eugene-Springfield, OR Metro Area_Value",error,97324_Value,...,97489_Value,error,97490_Value,error,97492_Value,error,97493_Value,error,97498_Value,error
0,Total:,127482864,208633,1701548,3556,159509,808,159509,808,474,...,409,147,133,68,483,240,213,58,1021,180
1,"Less than $10,000",6214601,26414,74987,2256,9187,862,9187,862,12,...,17,17,0,13,3,5,3,5,39,33
2,"$10,000 to $14,999",4531954,20357,57467,2065,6701,637,6701,637,24,...,0,13,0,13,11,10,0,13,32,30
3,"$15,000 to $19,999",3984613,16995,52738,1874,5798,626,5798,626,0,...,20,25,0,13,3,3,2,4,54,42
4,"$20,000 to $24,999",4386074,17832,54087,1981,5854,488,5854,488,4,...,0,13,0,13,27,42,22,21,67,38


In [19]:
income_data_clean = income_data[["unnamed: 0_level_1", "97402_Value", "97401_Value", "97403_Value", "97404_Value", "97405_Value", "97408_Value", 
"97477_Value", "97424_Value", "97448_Value", "97437_Value", "97419_Value"]]

income_data_clean = income_data_clean.rename(columns={"unnamed: 0_level_1": "income"})
income_data_clean.head()

,income,97402_Value,97401_Value,97403_Value,97404_Value,97405_Value,97408_Value,97477_Value,97424_Value,97448_Value,97437_Value,97419_Value
0,Total:,23077,22990,4714,13164,19588,6341,16178,7428,5915,927,375
1,"Less than $10,000",1374,2435,574,526,633,153,990,388,209,59,18
2,"$10,000 to $14,999",1273,1481,236,331,587,114,843,264,262,13,0
3,"$15,000 to $19,999",798,957,370,583,547,99,682,206,185,31,18
4,"$20,000 to $24,999",972,1098,167,444,712,99,569,274,258,9,23


In [20]:
race_dat_long = race_ethnicity_data_clean.melt(id_vars="race_and_ethnicity", var_name="zipcode", value_name="count_race")
race_dat_long["zipcode"] = race_dat_long["zipcode"].str.replace("_Value", "").astype(int)

sex_age_dat_long = sex_age_data_clean.melt(id_vars="sex_and_age", var_name="zipcode", value_name="count_sex_age")
sex_age_dat_long["zipcode"] = sex_age_dat_long["zipcode"].str.replace("_Value", "").astype(int)

education_dat_long = education_data_clean.melt(id_vars="education", var_name="zipcode", value_name="count_education")
education_dat_long["zipcode"] = education_dat_long["zipcode"].str.replace("_Value", "").astype(int)

income_dat_long = income_data_clean.melt(id_vars="income", var_name="zipcode", value_name="count_income")
income_dat_long["zipcode"] = income_dat_long["zipcode"].str.replace("_Value", "").astype(int)

In [21]:
race_dat_wide = race_dat_long.pivot_table(index="zipcode", columns="race_and_ethnicity", values="count_race", aggfunc = "sum").reset_index()

sex_age_dat_wide = sex_age_dat_long.pivot_table(index="zipcode", columns="sex_and_age", values="count_sex_age", aggfunc = "sum").reset_index()

education_dat_wide = education_dat_long.pivot_table(index="zipcode", columns="education", values="count_education", aggfunc = "sum").reset_index()

income_dat_wide = income_dat_long.pivot_table(index="zipcode", columns="income", values="count_income", aggfunc = "sum").reset_index()

In [22]:
race_dat_wide = race_dat_wide.drop(["Total:"], axis=1)
sex_age_dat_wide = sex_age_dat_wide.drop(["Total:"], axis=1)
education_dat_wide = education_dat_wide.drop(["Total:"], axis=1)
income_dat_wide = income_dat_wide .drop(["Total:"], axis=1)

In [23]:
combined_df = [race_dat_wide, sex_age_dat_wide, education_dat_wide, income_dat_wide]

demographic_dat = combined_df[0]
for i in range(1, len(combined_df)):
    demographic_dat = pd.merge(demographic_dat, combined_df[i], on="zipcode", how="outer")

In [24]:
demographic_dat.head()

,zipcode,American Indian and Alaska Native alone,Asian alone,Black or African American alone,Hispanic or Latino:,Native Hawaiian and Other Pacific Islander alone,Not Hispanic or Latino:,Some other race alone,Two or more races:,"Two races excluding Some other race, and three or more races",...,"$200,000 or more","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","Less than $10,000"
0,97401,258,2664,761,4748,165,43438,1771,4792,3194,...,1678,960,777,1200,1223,1171,1878,1476,2635,2435
1,97402,520,1228,1196,9013,118,45186,3290,6171,4163,...,560,1156,1103,958,1647,782,2317,2261,3100,1374
2,97403,104,814,349,1261,76,13247,521,1541,1094,...,461,305,179,218,255,107,272,276,296,574
3,97404,408,726,443,3250,164,30106,700,3025,2113,...,838,373,239,345,506,484,1011,1339,1984,526
4,97405,285,1352,325,3500,186,43370,1076,4472,2336,...,3053,637,452,612,383,668,1229,1380,2816,633


In [25]:
call_num_by_zip = call_data_clean.value_counts(["Call_Zipcode"]).reset_index()
call_num_by_zip.columns = ["zipcode", "num_calls"]
call_num_by_zip

,zipcode,num_calls
0,97401,34904
1,97402,32420
2,97405,6044
3,97404,3443
4,97403,1898
5,97408,1221
6,97477,22
7,97424,2
8,97448,2
9,97419,1


In [26]:
full_df = pd.merge(demographic_dat, call_num_by_zip, on="zipcode", how="left")

In [27]:
Eugene_zips = {97401, 97402, 97403, 97404, 97405, 97408, 97455, 97412, 97448, 97437, 97419}
Springfield_zips = {97477, 97478, 97424}

def eugene_or_springfield(zipcode):
    if zipcode in Eugene_zips:
        return "Eugene"
    elif zipcode in Springfield_zips:
        return "Springfield"
    else:
        return "Other"

full_df["city"] = full_df["zipcode"].apply(eugene_or_springfield)

In [28]:
full_df.to_csv("data/full_df")